![Hugging Face](https://miro.medium.com/max/2000/1*Z4mGaMsu34LfyE76QAi9qA.png)


# Using HuggingFace

In this tutorial, we will learn how to use the various functionalities offered by [HuggingFace](https://huggingface.co/).

## About Hugging Face

- HuggingFace is 'On a mission to solve NLP, One commit at a time', as per their tagline. 

- The HuggingFace Transformer library is closing on 26K Stars on GitHub now and provides state-of-the-art Transformer Based Models, their pretrained weights and a lots more (as we will see today)

- They recently released their Tokenisers library


They have originally used Rust, so that's an added advantage.

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.0MB 10.7MB/s 
     |████████████████████████████████| 3.2MB 35.9MB/s 
     |████████████████████████████████| 890kB 35.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=7fc5e8dd1e0457686d2bd8921fc28409bb87ec717d0da7cb347aa3a14cfb426a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Pipeline

https://github.com/huggingface/transformers#quick-tour-of-pipelines

In [ ]:
from transformers import pipeline

## Sentiment Analysis

In [ ]:
nlp = pipeline('sentiment-analysis')
nlp('We are very happy to include pipeline into the transformers repository.')

[{'label': 'POSITIVE', 'score': 0.9978193640708923}]

## Question Answering

In [ ]:
nlp = pipeline('question-answering')
nlp({
    'question': 'What is my name ?',
    'context': 'My name is Rachit, I work at HuggingFace'
})

{'answer': 'Rachit', 'end': 17, 'score': 0.9968027472496033, 'start': 11}

## Predicting Masks

In [ ]:
nlp = pipeline('fill-mask')
nlp('I hope you <mask> this video')

[{'score': 0.6921412944793701,
  'sequence': '<s> I hope you enjoyed this video</s>',
  'token': 3776},
 {'score': 0.15966762602329254,
  'sequence': '<s> I hope you enjoy this video</s>',
  'token': 2254},
 {'score': 0.12923917174339294,
  'sequence': '<s> I hope you liked this video</s>',
  'token': 6640},
 {'score': 0.005954346619546413,
  'sequence': '<s> I hope you like this video</s>',
  'token': 101},
 {'score': 0.004122701473534107,
  'sequence': '<s> I hope you appreciated this video</s>',
  'token': 10874}]

## NER

In [ ]:
nlp = pipeline('ner')
nlp('It is me, Rachit, I work at HuggingFace')

[{'end': 12,
  'entity': 'I-PER',
  'index': 5,
  'score': 0.9992461800575256,
  'start': 10,
  'word': 'Ra'},
 {'end': 15,
  'entity': 'I-PER',
  'index': 6,
  'score': 0.988210916519165,
  'start': 12,
  'word': '##chi'},
 {'end': 16,
  'entity': 'I-PER',
  'index': 7,
  'score': 0.991302490234375,
  'start': 15,
  'word': '##t'},
 {'end': 30,
  'entity': 'I-ORG',
  'index': 12,
  'score': 0.9990648031234741,
  'start': 28,
  'word': 'Hu'},
 {'end': 35,
  'entity': 'I-ORG',
  'index': 13,
  'score': 0.9934505820274353,
  'start': 30,
  'word': '##gging'},
 {'end': 36,
  'entity': 'I-ORG',
  'index': 14,
  'score': 0.9981080889701843,
  'start': 35,
  'word': '##F'},
 {'end': 39,
  'entity': 'I-ORG',
  'index': 15,
  'score': 0.9968755841255188,
  'start': 36,
  'word': '##ace'}]

In [ ]:
nlp.model.save_pretrained('...')

# Text Generation

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
tokeniser = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

## Single Word Prediction

In [ ]:
text = "let us see how this turns"
indexed_tokens = tokeniser.encode(text)
tokens_tensor = torch.tensor([indexed_tokens])
model.eval()
tokens_tensor = tokens_tensor.to('cuda')
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [ ]:
with torch.no_grad():
  outputs = model(tokens_tensor)
  predictions = outputs[0]

print(outputs[0].shape)

predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokeniser.decode(indexed_tokens + [predicted_index])

print(predicted_text)

torch.Size([1, 6, 50257])
let us see how this turns out


## Looping for multi-word

In [ ]:
chars = 0
text = "i am very exited to present to you this"
while chars<50:
  chars += 1
  indexed_tokens = tokeniser.encode(text)
  tokens_tensors = torch.tensor([indexed_tokens])
  tokens_tensors = tokens_tensors.to('cuda')
#  model = model.to('cuda')
  with torch.no_grad():
    outputs = model(tokens_tensors)
    predictions = outputs[0]
  predicted_index = torch.argmax(predictions[0,-1,:]).item()
  text = tokeniser.decode(indexed_tokens + [predicted_index])

print(text)

i am very exited to present to you this new book. I am very excited to share with you the first chapter of the book, "The Secret of the Soul."



## OR

In [ ]:
!git clone https://github.com/huggingface/pytorch-transformers.git

fatal: destination path 'pytorch-transformers' already exists and is not an empty directory.


In [ ]:
!python pytorch-transformers/examples/text-generation/run_generation.py \
    --model_type=gpt2 \
    --length=100 \
    --model_name_or_path=gpt2 \

2020-05-21 09:23:09.863295: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
05/21/2020 09:23:12 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /root/.cache/torch/transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
05/21/2020 09:23:12 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at /root/.cache/torch/transformers/d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
05/21/2020 09:23:12 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /root/.cache/torch/transformers/4b

# Summarising text using HuggingFace

In [ ]:
text = 'Shakespeare occupies a position unique in world literature. Other poets, such as Homer and Dante, and novelists, such as Leo Tolstoy and Charles Dickens, have transcended national barriers, but no writer’s living reputation can compare to that of Shakespeare, whose plays, written in the late 16th and early 17th centuries for a small repertory theatre, are now performed and read more often and in more countries than ever before. The prophecy of his great contemporary, the poet and dramatist Ben Jonson, that Shakespeare “was not of an age, but for all time,” has been fulfilled. It may be audacious even to attempt a definition of his greatness, but it is not so difficult to describe the gifts that enabled him to create imaginative visions of pathos and mirth that, whether read or witnessed in the theatre, fill the mind and linger there. He is a writer of great intellectual rapidity, perceptiveness, and poetic power. Other writers have had these qualities, but with Shakespeare the keenness of mind was applied not to abstruse or remote subjects but to human beings and their complete range of emotions and conflicts. Other writers have applied their keenness of mind in this way, but Shakespeare is astonishingly clever with words and images, so that his mental energy, when applied to intelligible human situations, finds full and memorable expression, convincing and imaginatively stimulating. As if this were not enough, the art form into which his creative energies went was not remote and bookish but involved the vivid stage impersonation of human beings, commanding sympathy and inviting vicarious participation. Thus, Shakespeare’s merits can survive translation into other languages and into cultures remote from that of Elizabethan England.'

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('bart-large-cnn')

In [ ]:
inputs = tokenizer.batch_encode_plus([text], max_length=1024, return_tensors='pt')

summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=100, early_stopping=False)

for ids in summary_ids:
    short = tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

    print(len(text), len(short))
    print(short)

1761 341
Shakespeare occupies a position unique in world literature. He is a writer of great intellectual rapidity, perceptiveness, and poetic power. Other writers have had these qualities, but with Shakespeare the keenness of mind was applied not to abstruse or remote subjects but to human beings and their complete range of emotions and conflicts.


HuggingFace Tranformers: https://github.com/huggingface/transformers

BART: https://arxiv.org/abs/1910.13461

Curious Case of Neural Text Degeneration: https://arxiv.org/abs/1904.09751